In [1]:
!pip install -q -U numpy matplotlib plotly pandas scipy scikit-learn openai python-dotenv

In [2]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
text_loader = TextFileLoader("data/KingLear.txt")
documents = text_loader.load_documents()
len(documents)

1

In [5]:
print(documents[0][:100])

ACT I
SCENE I. King Lear's palace.
Enter KENT, GLOUCESTER, and EDMUND
KENT
I thought the king had m


In [6]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

189

In [7]:
split_documents[0:1]

["\ufeffACT I\nSCENE I. King Lear's palace.\nEnter KENT, GLOUCESTER, and EDMUND\nKENT\nI thought the king had more affected the Duke of\nAlbany than Cornwall.\nGLOUCESTER\nIt did always seem so to us: but now, in the\ndivision of the kingdom, it appears not which of\nthe dukes he values most; for equalities are so\nweighed, that curiosity in neither can make choice\nof either's moiety.\nKENT\nIs not this your son, my lord?\nGLOUCESTER\nHis breeding, sir, hath been at my charge: I have\nso often blushed to acknowledge him, that now I am\nbrazed to it.\nKENT\nI cannot conceive you.\nGLOUCESTER\nSir, this young fellow's mother could: whereupon\nshe grew round-wombed, and had, indeed, sir, a son\nfor her cradle ere she had a husband for her bed.\nDo you smell a fault?\nKENT\nI cannot wish the fault undone, the issue of it\nbeing so proper.\nGLOUCESTER\nBut I have, sir, a son by order of law, some year\nelder than this, who yet is no dearer in my account:\nthough this knave came something s

In [8]:
import os
import openai
from getpass import getpass

In [9]:
openai.api_key = getpass("OpenAI API Key: ")

In [10]:
os.environ["OPENAI_API_KEY"] = openai.api_key

In [11]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

In [12]:
vector_db.search_by_text("Your servant Kent. Where is your servant Caius?", k=3)

[("ng] O my good master!\nKING LEAR\nPrithee, away.\nEDGAR\n'Tis noble Kent, your friend.\nKING LEAR\nA plague upon you, murderers, traitors all!\nI might have saved her; now she's gone for ever!\nCordelia, Cordelia! stay a little. Ha!\nWhat is't thou say'st? Her voice was ever soft,\nGentle, and low, an excellent thing in woman.\nI kill'd the slave that was a-hanging thee.\nCaptain\n'Tis true, my lords, he did.\nKING LEAR\nDid I not, fellow?\nI have seen the day, with my good biting falchion\nI would have made them skip: I am old now,\nAnd these same crosses spoil me. Who are you?\nMine eyes are not o' the best: I'll tell you straight.\nKENT\nIf fortune brag of two she loved and hated,\nOne of them we behold.\nKING LEAR\nThis is a dull sight. Are you not Kent?\nKENT\nThe same,\nYour servant Kent: Where is your servant Caius?\nKING LEAR\nHe's a good fellow, I can tell you that;\nHe'll strike, and quickly too: he's dead and rotten.\nKENT\nNo, my good lord; I am the very man,--\nKING LEA

In [13]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

In [14]:
chat_openai = ChatOpenAI()

In [19]:
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
user_role_prompt.get_input_variables()

['content']

In [20]:
user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    )

{'role': 'user', 'content': 'What is the best way to write a loop?'}

In [21]:
system_prompt_template = "You are an expert in {expertise}, you always answer in a kind way."
system_role_prompt = SystemRolePrompt(system_prompt_template)
system_role_prompt.get_input_variables()

['expertise']

In [22]:
system_role_prompt.create_message(expertise="Python")

{'role': 'system',
 'content': 'You are an expert in Python, you always answer in a kind way.'}

In [23]:
messages = [
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
    system_role_prompt.create_message(expertise="Python"),
]
messages

[{'role': 'user', 'content': 'What is the best way to write a loop?'},
 {'role': 'system',
  'content': 'You are an expert in Python, you always answer in a kind way.'}]

In [25]:
from openai import AsyncOpenAI

In [29]:
client = AsyncOpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [31]:
async def main():
    stream = await client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": "Say this is a test"}],
        stream=True,
    )
    async for chunk in stream:

        #print(chunk.choices[0].delta.content or "", end="")

        token = chunk.choices[0].delta.content
        if not token:
            token = ""
        print(token)


asyncio.run(main())


This
 is
 a
 test
.



In [32]:
async def main():
    stream = await client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": "Say this is a test"}],
        stream=True,
    )
    async for chunk in stream:

        print(chunk.choices[0].delta.content or "", end="")


asyncio.run(main())

This is a test.